# SISTEMA DE RECOMENDAÇÃO DE FILMES

## A. Introdução
• Detalhar objetivo escolhido para trabalhar neste projeto juntamente com descrição da
base de dados. Pesquise trabalhos na literatura que discutam o tema escolhido. Para
trabalhos acadêmicos, acesse https://scholar.google.com.br/. Guarde as referências
estudadas para citá-las no projeto.


## B. Minerando Dados e Características do Dataset
• Se necessário, faça filtro na base de dados tanto de linhas como de colunas em prol do
objetivo traçado anteriormente.
• Descreva as variáveis finais que serão utilizadas a partir deste ponto.
• Faça análise descritiva detalhada das variáveis, norteado pelo objetivo do problema.


## C. Modelos de Predição
• Descreva e justifique sua escolha de modelos de predição. Exemplos de modelos (os
exemplos a seguir são utilizados neste trabalho, mas você pode usar outros que fizerem
mais sentido para o seu problema):
o Modelo de predição pela média (sem uso de variável explicativa);
o Modelo de regressão linear (Multiple Linear Regression);
o Modelo de árvores de regressão (Decision Tree Regression).


## D. Processo e Estatísticas de Validação
• Para os modelos preditivos que foram desenvolvidos no item anterior, é necessário
calcular medidas que informam a performance de cada modelo ajustado. Assim, para
cada modelo preditivo, faça:
o Divida a base de dados na parte treinamento e na parte teste. Use a parte
treinamento para estimar cada modelo preditivo.
o Estude as medidas coeficiente de determinação (R2) e raiz do erro médio
quadrático (RMS) descritas aqui. Calcule essas medidas tanto para a parte dos
dados de treinamento como para a parte dos dados teste.
o Discuta se essas duas medidas se comportam de forma semelhante para as duas
partes de dados. Leia o texto disponível aqui para compreender overfitting e
underfitting e, com isso, refinar senso crítico para discutir sobre as medidas
calculadas.
o Extra: Faça o processo de Validação Cruzada utilizando também 10 ciclos e
calcule a performance média e desvio padrão das duas medidas R2 e RMS tanto
para a parte treinamento como para a parte teste. Discuta com riqueza de
detalhes.


## E. Conclusão
• Faça conclusão final com detalhes levando em consideração todas as interpretações
realizadas no decorrer do projeto.


## F. Referências Bibliográficas
• Todas as pesquisas feitas e estudadas que foram relevantes para o desenvolvimento
devem ser citadas no projeto.